In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/"Voice Swapper"/cmu_us_jmk_arctic

/content/drive/MyDrive/Voice Swapper/cmu_us_jmk_arctic


In [ ]:
!pip install wav2mel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!wav2mel /wav/arctic_a0001.wav

/bin/bash: wav2mel: command not found


In [ ]:
import torch
import torchvision
import torch.nn as nn
import math

In [ ]:
import math

import torch
import torchvision
import torch.nn as nn


class Discriminator(nn.Module):
  def __init__(self,input_nc=3, filters=64, n_layers=0):
    netD = []
        
    # input is (nc) x 256 x 256
    netD.append(nn.Conv2d(input_nc, filters, kernel_size=4,stride=2))
    netD.append(nn.LeakyReLU(0.2, True))
    
    nf_mult = 1
    nf_mult_prev = 1
    for n in range(1,n_layers):
      nf_mult_prev = nf_mult
      nf_mult = math.min(2^n,8)
      netD.append(nn.Conv2d(filters * nf_mult_prev, filters * nf_mult, kernel_size=4, stride=2))
      netD.append(nn.BatchNorm2d(filters * nf_mult))
      netD.append(nn.LeakyReLU(0.2, True))
    
    # state size: (filters*M) x N x N
    nf_mult_prev = nf_mult
    nf_mult = math.min(2^n_layers,8)
    netD.append(nn.Conv2d(filters * nf_mult_prev, filters * nf_mult, kernel_size=4,stride=1))
    netD.append(nn.BatchNorm2d(filters * nf_mult))
    netD.append(nn.LeakyReLU(0.2, True))
    
    # state size: (filters*M*2) x (N-1) x (N-1)
    netD.append(nn.Conv2d(filters * nf_mult, 1, kernel_size=4,stride=1))
    # state size: 1 x (N-2) x (N-2)
    
    netD.append(nn.Sigmoid())
    # state size: 1 x (N-2) x (N-2)
    
    self.model = nn.Sequential(*netD)

  def forward(self,x):
    return self.model(x)

In [ ]:
""" Parts of the U-Net model """

import torch
import torch.nn as nn
import torch.nn.functional as F


class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Encode(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Decode(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        # if you have padding issues, see
        # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
        # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        
    def forward(self, x):
        return self.conv(x)

In [ ]:
from .unet_parts import *

class Generator(nn.Module):
  def __init__(self, in_channels, out_channels, bilinear=False):
    super().__init__()

    self.in_channels = in_channels
    self.out_channels = out_channels
    self.bilinear = bilinear

    # self.inc = (DoubleConv(in_channels, 64))
    self.encode1 = (Encode(in_channels, 64))
    self.encode2 = (Encode(64, 128))
    self.encode3 = (Encode(128, 256))
    factor = 2 if bilinear else 1
    self.encode4 = (Encode(256, 512 // factor))
    self.decode1 = (Decode(512, 256 // factor, bilinear))
    self.decode2 = (Decode(256, 128 // factor, bilinear))
    self.decode3 = (Decode(128, 64 // factor, bilinear))
    self.decode4 = (Decode(64, out_channels, bilinear))
    self.out = nn.Tanh()
    # self.outc = (OutConv(64, out_channels))
  
  def forward(self, x): #x is noise
    x_l1,x_l2 = np.split(x,2,axis=1)

    x2 = self.encode1(x_l1)
    x3 = self.encode2(x2)
    x4 = self.encode3(x3)
    x5 = self.encode4(x4)
    x = self.decode1(x5, x4)
    x = self.decode2(x, x3)
    x = self.decode3(x, x2)
    x = self.decode4(x, x1)
    output_l1 = self.outc(x)

    x2 = self.encode1(x_l2)
    x3 = self.encode2(x2)
    x4 = self.encode3(x3)
    x5 = self.encode4(x4)
    x = self.decode1(x5, x4)
    x = self.decode2(x, x3)
    x = self.decode3(x, x2)
    x = self.decode4(x, x1)
    output_l2 = self.outc(x)

    return np.concat((output_l1,output_l2),axis=1)

In [ ]:
in_channels = MEL_CHANNELS
out_channels = MEL_CHANNELS

netG = Generator(in_channels,out_channels)
netD = Discriminator()
fft = Audio2Mel()

In [ ]:
import torch.optim as optim

criterion = nn.BCELoss()
optimizerG = optim.Adam(netG.parameters(), lr=1e-4)
optimizerD = optim.Adam(netD.parameters(), lr=4e-4)

In [ ]:

for epoch in range(NUM_EPOCHS):
  for i,real_audio in enumerate(dataloader):
    # convert to spec
    x = fft(real_audio)
    # get generated spec
    x_pred = netG(x)
    # get output of discriminator on real spectrogram
    netD.zero_grad()
    source_spec,target_spec = data[0],data[1]
    output = netD(target_spec)
    errD_real = criterion(output,1) #assume 1 is real and 0 is fake
    errD_real.backward()
    D_x = output.mean().item()

    # train generator
    fake = netG(source_spec)
    output = netD(fake.detach()).view(-1)
    errD_fake = criterion(output,0)
    errD_fake.backward()
    D_G_z1 = output.mean().item()

    # Compute error of D as sum over the fake and the real batches
    errD = errD_real + errD_fake

    optimizerD.step()

    ############################
    # (2) Update G network: maximize log(D(G(z)))
    ###########################
    netG.zero_grad()
    label.fill_(real_label)  # fake labels are real for generator cost
    # Since we just updated D, perform another forward pass of all-fake batch through D
    output = netD(fake).view(-1)
    # Calculate G's loss based on this output
    errG = criterion(output, label)
    # Calculate gradients for G
    errG.backward()
    D_G_z2 = output.mean().item()
    # Update G
    optimizerG.step()